# **Kaggle reproduce**

## Ismertető, előkészítés, előfeldolgozás

Használt modell és algoritmus:


*   Logistic Regression
*   n-gram with n = 1, 2, 3
*   Text preprocces
*   News merge into one for one day



Forrás: https://www.kaggle.com/pengzihao917/86-accuracy-with-n-gram

Található benne saját munka:


*   A magyarázatok
*   Ellenőrző mechanizmusok és algoritmusok
*   A LogisticRegression kiegészítése a deprecated forrás miatt
*   Kiegészítés 4-gram és 5-gra modellekkel és eredményeikkel



Alapvetően hibás eredményeket fedeztem fel a téves adathalmaz felosztás miatt, de azokat orvosoltam és a jó megoldás eredményei láthatóak a továbbiakban az ő algoritmusai alapján.

Eredmények:


*   1-gram modell --- 0.43915343915343913
*   2-gram modell --- 0.47619047619047616
*   3-gram modell --- 0.5555555555555556 (range: 2, 3)
*   4-gram modell --- 0.5396825396825397
*   5-gram modell ---0.5264550264550265





A range-k minden lehetséges kombinációját próbálgattam az adott n-gram modell esetében és a maximálisat írtam fel. (Értelemszerűen változik a random seed miatt is, de azt fixáltam, így ez a notebook reprodukálható teljes mértékben.

A Drive-omról betöltöm a használt adathalmazt.

In [1]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

# Copy the dataset to the local environment
!cp "/content/drive/MyDrive/Combined_News_DJIA.csv" "Combined_News_DJIA.csv"

# Check the copy is succesfull -> good if no assertation error
read = !ls
assert read[0].find("Combined_News_DJIA.csv") != -1

Mounted at /content/drive


A szükséges könyvtárak betöltése a projekthez.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

A reprodukálhatóság miatt definiálok egy seed-et a véletlen szám generátorhoz, amit a továbbiakban használni fogok.

In [3]:
# Random seed
RANDOM_SEED = 1234

# Max iteration for training
MAX_ITER = 100000

Az adathalmaz beolvasása.

In [4]:
data = pd.read_csv('Combined_News_DJIA.csv')
data.head(1)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",b'Georgian troops retreat from S. Osettain cap...,b'Did the U.S. Prep Georgia for War with Russia?',b'Rice Gives Green Light for Israel to Attack ...,b'Announcing:Class Action Lawsuit on Behalf of...,"b""So---Russia and Georgia are at war and the N...","b""China tells Bush to stay out of other countr...",b'Did World War III start today?',b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""


Az adathalmaz szétbontása tanító és validáló/tesztelő adathalmazokra, majd a szétbontás ellenőrzése mérettel és első elem kiíratásával.

In [5]:
# Split the dataset
train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] > '2014-12-31']

# Print out the length
print("Train data set length: " + str(len(train)))
print("Test data set length: " + str(len(test)))
print("Split summa: " + str(len(train) + len(test)))
print("Dataset summa before split: " + str(len(data)))

# check
split_sum = len(train) + len(test)
sum = len(data)
assert split_sum == sum

Train data set length: 1611
Test data set length: 378
Split summa: 1989
Dataset summa before split: 1989


In [6]:
train.head(1)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",b'Georgian troops retreat from S. Osettain cap...,b'Did the U.S. Prep Georgia for War with Russia?',b'Rice Gives Green Light for Israel to Attack ...,b'Announcing:Class Action Lawsuit on Behalf of...,"b""So---Russia and Georgia are at war and the N...","b""China tells Bush to stay out of other countr...",b'Did World War III start today?',b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""


In [7]:
test.head(1)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
1611,2015-01-02,1,Most cases of cancer are the result of sheer b...,Iran dismissed United States efforts to fight ...,Poll: One in 8 Germans would join anti-Muslim ...,UK royal family's Prince Andrew named in US la...,Some 40 asylum-seekers refused to leave the bu...,Pakistani boat blows self up after India navy ...,Sweden hit by third mosque arson attack in a week,940 cars set alight during French New Year,Salaries for top CEOs rose twice as fast as av...,"Norway violated equal-pay law, judge says: Jud...",Imam wants radical recruiters of Muslim youth ...,"Saudi Arabia beheaded 83 people in 2014, the m...",'A living hell' for slaves on remote South Kor...,"Worlds 400 richest get richer, adding $92bn in...","Rental Car Stereos Infringe Copyright, Music R...",Ukrainian minister threatens TV channel with c...,Palestinian President Mahmoud Abbas has entere...,Israeli security center publishes names of 50 ...,The year 2014 was the deadliest year yet in Sy...,A Secret underground complex built by the Nazi...,Restrictions on Web Freedom a Major Global Iss...,Austrian journalist Erich Mchel delivered a pr...,Thousands of Ukraine nationalists march in Kiev,Chinas New Years Resolution: No More Harvestin...,Authorities Pull Plug on Russia's Last Politic...


A szöveg előfeldolgozása következik, amelyek során az írásjeleket távolítom el, illetve minden szót kisbetűs szóvá konvertálok.

In [8]:
# Removing punctuations
slicedData= train.iloc[:,2:27]
slicedData.replace(to_replace="[^a-zA-Z]", value=" ", regex=True, inplace=True)

# Renaming column names for ease of access
list1= [i for i in range(25)]
new_Index=[str(i) for i in list1]
slicedData.columns= new_Index
slicedData.head(5)

# Converting headlines to lower case
for index in new_Index:
    slicedData[index]=slicedData[index].str.lower()
slicedData.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,b georgia downs two russian warplanes as cou...,b breaking musharraf to be impeached,b russia today columns of troops roll into so...,b russian tanks are moving towards the capital...,b afghan children raped with impunity u n ...,b russian tanks have entered south ossetia...,b breaking georgia invades south ossetia rus...,b the enemy combatent trials are nothing but...,b georgian troops retreat from s osettain cap...,b did the u s prep georgia for war with russia,b rice gives green light for israel to attack ...,b announcing class action lawsuit on behalf of...,b so russia and georgia are at war and the n...,b china tells bush to stay out of other countr...,b did world war iii start today,b georgia invades south ossetia if russia ge...,b al qaeda faces islamist backlash,b condoleezza rice the us would not act to p...,b this is a busy day the european union has ...,b georgia will withdraw soldiers from ir...,b why the pentagon thinks attacking iran is a ...,b caucasus in crisis georgia invades south os...,b indian shoe manufactory and again in a se...,b visitors suffering from mental illnesses ban...,b no help for mexico s kidnapping surge


Az adott naphoz tartozó híreket fűzöm össze egy sztringbe, hogy a következőkben ez alkothassa a modellem bemenetét.

In [9]:
# Merge the headlines
headlines = []
for row in range(0,len(slicedData.index)):
    headlines.append(' '.join(str(x) for x in slicedData.iloc[row,0:25]))

# Show it
headlines[0]

'b georgia  downs two russian warplanes  as countries move to brink of war  b breaking  musharraf to be impeached   b russia today  columns of troops roll into south ossetia  footage from fighting  youtube   b russian tanks are moving towards the capital of south ossetia  which has reportedly been completely destroyed by georgian artillery fire  b afghan children raped with  impunity   u n  official says   this is sick  a three year old was raped and they do nothing  b     russian tanks have entered south ossetia whilst georgia shoots down two russian jets   b breaking  georgia invades south ossetia  russia warned it would intervene on so s side  b the  enemy combatent  trials are nothing but a sham  salim haman has been sentenced to       years  but will be kept longer anyway just because they feel like it   b georgian troops retreat from s  osettain capital  presumably leaving several hundred people killed   video   b did the u s  prep georgia for war with russia   b rice gives green

## 1-gram model

Először az 1-gram modellem megalkotása következik, majd az összefűzött híreket ennek segítségével vektorizálom. Végül kiírom a vektorizált adathalmaz dimenzióját.

In [10]:
basicvectorizer = CountVectorizer(ngram_range=(1,1))
basictrain = basicvectorizer.fit_transform(headlines)
print(basictrain.shape)

(1611, 30442)


Ezek után a klasszifikáló modellem definiálom, ami jelen esetben egy Logistic Regression modell. A betanítása következik utána.

In [11]:
# model 1: logistic regression
# 1-gram
basicmodel = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
basicmodel = basicmodel.fit(basictrain, train["Label"])

A következőkben a teszt adathalmaz azonos napi híreinek az összefűzése történik egy sztringbe, majd azokat egy tömbbe helyezem. Ezek után ezt is a korábbi módszerrel vektorizálom, majd végrehajtom az előrejelzést és kiíratom annak az eredményt a tömb elemeinek megfelelően.

In [12]:
# Create test array
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))

# Vectorize the test dataset    
basictest = basicvectorizer.transform(testheadlines)

# Make predictions
predictions = basicmodel.predict(basictest)

# Print it out
predictions

array([0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,

A következőkben az előrejelzés pontosságát vizsgálom.

In [13]:
pd.crosstab(test["Label"], predictions, rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,63,123
1,89,103


In [14]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix

print (classification_report(test["Label"], predictions))
print (accuracy_score(test["Label"], predictions))

              precision    recall  f1-score   support

           0       0.41      0.34      0.37       186
           1       0.46      0.54      0.49       192

    accuracy                           0.44       378
   macro avg       0.44      0.44      0.43       378
weighted avg       0.44      0.44      0.43       378

0.43915343915343913


## 2-gram modell

A következőkben a korábbiakat végzem el, csak 2-gram modell esetében. Nem fűzök majd hozzá további magyarázatokat.

In [15]:
# 2-gram
basicvectorizer2 = CountVectorizer(ngram_range=(1,2))
basictrain2 = basicvectorizer2.fit_transform(headlines)
print(basictrain2.shape)

basicmodel2 = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
basicmodel2 = basicmodel2.fit(basictrain2, train["Label"])

basictest2 = basicvectorizer2.transform(testheadlines)
predictions2 = basicmodel2.predict(basictest2)

pd.crosstab(test["Label"], predictions2, rownames=["Actual"], colnames=["Predicted"])

print (classification_report(test["Label"], predictions2))
print (accuracy_score(test["Label"], predictions2))

(1611, 388181)
              precision    recall  f1-score   support

           0       0.46      0.33      0.38       186
           1       0.49      0.62      0.55       192

    accuracy                           0.48       378
   macro avg       0.47      0.47      0.46       378
weighted avg       0.47      0.48      0.46       378

0.47619047619047616


## 3-gram modell

A következőkben a korábbiakat végzem el, csak 3-gram modell esetében. Nem fűzök majd hozzá további magyarázatokat.

In [16]:
# 3-gram
basicvectorizer3 = CountVectorizer(ngram_range=(2,3))
basictrain3 = basicvectorizer3.fit_transform(headlines)
print(basictrain3.shape)

basicmodel3 = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
basicmodel3 = basicmodel3.fit(basictrain3, train["Label"])

basictest3 = basicvectorizer3.transform(testheadlines)
predictions3 = basicmodel3.predict(basictest3)

pd.crosstab(test["Label"], predictions3, rownames=["Actual"], colnames=["Predicted"])

print (classification_report(test["Label"], predictions3))
print (accuracy_score(test["Label"], predictions3))

(1611, 957711)
              precision    recall  f1-score   support

           0       0.63      0.24      0.34       186
           1       0.54      0.86      0.66       192

    accuracy                           0.56       378
   macro avg       0.58      0.55      0.50       378
weighted avg       0.58      0.56      0.51       378

0.5555555555555556


## 4-gram modell

A következőkben a korábbiakat végzem el, csak 4-gram modell esetében. Nem fűzök majd hozzá további magyarázatokat.

In [17]:
# 4-gram
basicvectorizer4 = CountVectorizer(ngram_range=(2,4))
basictrain4 = basicvectorizer4.fit_transform(headlines)
print(basictrain4.shape)

basicmodel4 = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
basicmodel4 = basicmodel4.fit(basictrain4, train["Label"])

basictest4 = basicvectorizer4.transform(testheadlines)
predictions4 = basicmodel4.predict(basictest4)

pd.crosstab(test["Label"], predictions4, rownames=["Actual"], colnames=["Predicted"])

print (classification_report(test["Label"], predictions4))
print (accuracy_score(test["Label"], predictions4))

(1611, 1619668)
              precision    recall  f1-score   support

           0       0.65      0.14      0.23       186
           1       0.53      0.93      0.67       192

    accuracy                           0.54       378
   macro avg       0.59      0.53      0.45       378
weighted avg       0.59      0.54      0.45       378

0.5396825396825397


## 5-gram modell

A következőkben a korábbiakat végzem el, csak 5-gram modell esetében. Nem fűzök majd hozzá további magyarázatokat.

In [18]:
# 5-gram
basicvectorizer5 = CountVectorizer(ngram_range=(2,5))
basictrain5 = basicvectorizer5.fit_transform(headlines)
print(basictrain5.shape)

basicmodel5 = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
basicmodel5 = basicmodel5.fit(basictrain5, train["Label"])

basictest5 = basicvectorizer5.transform(testheadlines)
predictions5 = basicmodel5.predict(basictest5)

pd.crosstab(test["Label"], predictions5, rownames=["Actual"], colnames=["Predicted"])

print (classification_report(test["Label"], predictions5))
print (accuracy_score(test["Label"], predictions5))

(1611, 2292979)
              precision    recall  f1-score   support

           0       0.64      0.09      0.15       186
           1       0.52      0.95      0.67       192

    accuracy                           0.53       378
   macro avg       0.58      0.52      0.41       378
weighted avg       0.58      0.53      0.42       378

0.5264550264550265
